## **DEMO EXTRACCIÓN Y MODELADO DE DATOS DE ESTADOS FINANCIEROS**

**Nota Importante: Todos los ejemplos que vamos a ver a lo largo de este cuaderno son solo para fines educativos y no constituyen recomendación de inversión.**

En este cuaderno jupyter vamos a ver ejemplos de como descargar datos financieros de balance, cash flow y PyG de cualquier empresa cotizada, para ello utilizaremos la libreria **`yfinance`**. 

En próximas sesiones repetiremos estos ejercicios, pero lo haremos con un bucle (for) en el que descargaremos los datos ticker por ticker e iremos creando una tabla o dataframe (df) en el que iremos concatenando los datos que extrayamos tras cada iteración.

El bucle funcionará de la siguiente manera:
0. Paso preliminar: creamos un df vacio en el que iremos añadiendo los datos de cada ticker.
1. Iteraremos en cada ticker del listado de empresas cotizadas del S&P 500.
2. Extraera todos los datos financieros que le pidamos.
3. Una vez tengamos cada serie de datos (balance, cashflow, P&G), los añadiremos en la tabla hasta tener los datos financieros de todas las empresas cotizadas.

Importamos las librerias necesarias para este ejercicio

In [5]:
# Preparamos la celda para importar las librerias necesarias para el proyecto, en este caso pandas, numpy y yfinance. Si no se encuentran instaladas, se mostrará un mensaje de error indicando cuál es la librería faltante y cómo instalarla.

# Importamos las librerías necesarias
    # pandas: para manipulación y análisis de datos en forma de tablas (DataFrames)
    # requests: para hacer peticiones HTTP y descargar contenido de páginas web
    # yfinance: para obtener datos financieros desde Yahoo Finance

try:
    import pandas as pd
    import numpy as np
    import yfinance as yf
    print("se han importado las librerias correctamente")
except ImportError as e:
    print(f"Mensaje de error: {e}. Por favor, asegúrate de tener instaladas la libreria indicada (pip install nombre_de_libreria).")



se han importado las librerias correctamente


Recordad que debéis tener todas las librerias instaladas, acudid a la terminal o a Análisis de Sistema:

**`pip install nombre_de_libreria`** 

### **Datos de Balance, Cash Flow y P&G**

`yfinance` esta configurado de tal manera que el procedimiento para obtener cada una de estas series de datos es muy similar, unicamente debemos modificar una linea de codigo.

Los bucles que programaremos relizaran esta operación en todas y cada una de las empresas de nuestro listado.

In [6]:
#Ejemplo de cómo descargar los datos de balance de un ticker específico (por ejemplo, 'META' para Meta Platforms, Inc.)
ticker = 'TSLA'
#creamos la variable stock con el ticker especificado, usando yfinance para que la libreria pueda descargar los datos
stock = yf.Ticker(ticker)
#Dentro de cada iteración, vamos a descargar los datos de balance de cada ticker, en todos los años disponibles que tengamos y lo añadiremos a la tabla
# ESTA ES LA ÚNICA PARTE DEL CODIGO QUE CAMBIARÍA, podemos elegir entre financials, balance_sheet, cashflow, etc.
df_ticker = stock.balance_sheet # si quisieramos cambiar la series de datos. únicamrente cambiaríamos esta línea.
df_ticker.head()
### Podéis después exportar los datos a excel o csv si queréis trabajar con ellos fuera de Python. ###
# df_ticker.to_excel(f'datos_financieros_{ticker}.xlsx')

,2025-12-31,2024-12-31,2023-12-31,2022-12-31,2021-12-31
Treasury Shares Number,NaN,NaN,0.000000e+00,NaN,NaN
Ordinary Shares Number,3.751000e+09,3.216000e+09,3.185000e+09,3.164000e+09,NaN
Share Issued,3.751000e+09,3.216000e+09,3.185000e+09,3.164000e+09,NaN
Total Debt,1.471900e+10,1.362300e+10,9.573000e+09,5.748000e+09,NaN
Tangible Book Value,8.074800e+10,7.144300e+10,6.201900e+10,4.411100e+10,NaN


Ahora, debemos modelar estos datos para que estén estandarizados y organizados de manera que la IA los pueda procesar de manera más eficiente.

Por ejemplo, debemos trasponer las columnas de fecha y convertilas en atributos (de columans a filas), puesto que cada empresa tiene fechas de publicación distintas dentro del mismo año.

In [7]:
#trasponemos el dataframe para tener los años como filas y las cuentas como columnas (paso provisional)
df_ticker_t = df_ticker.T
# #Hacemos unpivot (melt) para tener una tabla larga con columnas: Año, Cuenta, Valor
df_ticker_unpiv = df_ticker_t.reset_index().melt(id_vars='index', var_name='Account', value_name='Value')  # Unpivoting
# print(df_ticker_unpiv['Account'].unique())
# #Añadimos una columna con el ticker para identificar cada empresa
df_ticker_unpiv['ticker'] = ticker
df_ticker_unpiv.head()
# df_ticker_unpiv

,index,Account,Value,ticker
0,2025-12-31,Treasury Shares Number,NaN,TSLA
1,2024-12-31,Treasury Shares Number,NaN,TSLA
2,2023-12-31,Treasury Shares Number,0.0,TSLA
3,2022-12-31,Treasury Shares Number,NaN,TSLA
4,2021-12-31,Treasury Shares Number,NaN,TSLA


**Simulación de bucle**

Crearemos un listado con solo dos empresas y haremos una muestra de como funcionara el bucle en cualquiera de las series de datos con las que trabajaremos 

Ademas de la descarga y el modelado, existe el ultimo paso que es fundamental. Necesitamos cargar los datos en el dataframe principal

Fijaos en el siguiente paso:
    
    df_tickers = pd.concat([df_tickers, df_ticker_unpiv], ignore_index=True)

Con cada iteración se cargaran nuevos datos en nuestro dataframe df_tickers, añadimos df_ticker_unpiv que contiene los datos modelados de la iteración que se este llevando a cabo.

In [8]:
# Definir el listado de 10 tickers (pueden ser los que queráis, aquí pongo dos de ejemplo)
tickers = ['AAPL','META','TSLA','AMZN','GOOGL','MSFT','NFLX','NVDA','INTC','IBM']

# df preliminar sin datos
df_tickers = pd.DataFrame()

#Abrimos el bucle para cada ticker
for ticker in tickers:
    print(f"Descargando datos para {ticker}...")
    # 1 DESCARGA DE DATOS PARA CADA TICKER
    stock = yf.Ticker(ticker) # ticker representa cada uno de los símbolos en la lista, que son dos.
    # Descargamos las series de datos para cada ticker y las almacenamos en una lista
    df_ticker = stock.cash_flow  # Cambiar aquí si se quieren otros datos
    # 2 MODELADO DE DATOS PARA CADA TICKER
    # Trasponemos el dataframe para tener los años como filas y las cuentas como columnas (paso provisional)
    df_ticker_t = df_ticker.T
    # Hacemos unpivot (melt) para tener una tabla larga con columnas: Año, Cuenta, Valor
    df_ticker_unpiv = df_ticker_t.reset_index().melt(id_vars='index', var_name='Account', value_name='Value')  # Unpivoting
    # Añadimos una columna con el ticker para identificar cada empresa
    df_ticker_unpiv['ticker'] = ticker
    # 3 CARGAMOS LOS DATOS DE CADA TICKER EN EL DATAFRAME PRINCIPALs
    # Concatenamos los datos al dataframe principal
    df_tickers = pd.concat([df_tickers, df_ticker_unpiv], ignore_index=True)

print(df_tickers['ticker'].unique())
df_tickers.head()

Descargando datos para AAPL...
Descargando datos para META...
Descargando datos para TSLA...
Descargando datos para AMZN...
Descargando datos para GOOGL...
Descargando datos para MSFT...
Descargando datos para NFLX...
Descargando datos para NVDA...
Descargando datos para INTC...
Descargando datos para IBM...
['AAPL' 'META' 'TSLA' 'AMZN' 'GOOGL' 'MSFT' 'NFLX' 'NVDA' 'INTC' 'IBM']


,index,Account,Value,ticker
0,2025-09-30,Free Cash Flow,9.876700e+10,AAPL
1,2024-09-30,Free Cash Flow,1.088070e+11,AAPL
2,2023-09-30,Free Cash Flow,9.958400e+10,AAPL
3,2022-09-30,Free Cash Flow,1.114430e+11,AAPL
4,2021-09-30,Free Cash Flow,NaN,AAPL
